In [2]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as pyplot
import seaborn as sns

In [3]:
netflix_dataset=pd.read_csv('/content/drive/MyDrive/Netflix_Dataset/combined_data_1.txt', header=None, names=['Cust_id','Ratings'], usecols=[0,1])

In [4]:
netflix_dataset

Cust_id  Ratings
0              1:      NaN
1         1488844      3.0
2          822109      5.0
3          885013      4.0
4           30878      4.0
...           ...      ...
24058258  2591364      2.0
24058259  1791000      2.0
24058260   512536      5.0
24058261   988963      3.0
24058262  1704416      3.0

[24058263 rows x 2 columns]

In [5]:
Movie_count=netflix_dataset['Ratings'].isnull().sum()
Movie_count

4499

In [6]:
Customer_count=netflix_dataset['Cust_id'].nunique()-Movie_count
Customer_count

470758

In [7]:
Total_Ratings=netflix_dataset['Cust_id'].count()-Movie_count
Total_Ratings

24053764

In [8]:
star_counts=netflix_dataset['Ratings'].value_counts()
star_counts

Ratings
4.0    8085741
3.0    6904181
5.0    5506583
2.0    2439073
1.0    1118186
Name: count, dtype: int64

In [9]:
movie_id=None
movie_np=[]

for customer in netflix_dataset['Cust_id']:
  if ':' in customer:
    movie_id=int(customer.replace(':',''))  # 1: -> 1

  movie_np.append(movie_id)


In [10]:
netflix_dataset['Movie_id']=movie_np

In [11]:
netflix_dataset

Cust_id  Ratings  Movie_id
0              1:      NaN         1
1         1488844      3.0         1
2          822109      5.0         1
3          885013      4.0         1
4           30878      4.0         1
...           ...      ...       ...
24058258  2591364      2.0      4499
24058259  1791000      2.0      4499
24058260   512536      5.0      4499
24058261   988963      3.0      4499
24058262  1704416      3.0      4499

[24058263 rows x 3 columns]

In [12]:
netflix_dataset.dropna(inplace=True)
netflix_dataset

Cust_id  Ratings  Movie_id
1         1488844      3.0         1
2          822109      5.0         1
3          885013      4.0         1
4           30878      4.0         1
5          823519      3.0         1
...           ...      ...       ...
24058258  2591364      2.0      4499
24058259  1791000      2.0      4499
24058260   512536      5.0      4499
24058261   988963      3.0      4499
24058262  1704416      3.0      4499

[24053764 rows x 3 columns]

In [13]:
netflix_dataset['Cust_id']=netflix_dataset['Cust_id'].astype(int)

In [14]:
netflix_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24053764 entries, 1 to 24058262
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Cust_id   int64  
 1   Ratings   float64
 2   Movie_id  int64  
dtypes: float64(1), int64(2)
memory usage: 734.1 MB


In [15]:
#movie_re_count=netflix_dataset['Movie_id'].value_counts()
movie_re_count=netflix_dataset.groupby('Movie_id')['Ratings'].agg('count')
movie_re_count

Movie_id
1        547
2        145
3       2012
4        142
5       1140
        ... 
4495     614
4496    9519
4497     714
4498     269
4499     428
Name: Ratings, Length: 4499, dtype: int64

In [16]:
bench_mark=round(movie_re_count.quantile(.6),0)

In [17]:
bench_mark # threshold of acceptance for model building

908.0

In [18]:
drop_movie_index=movie_re_count[movie_re_count<bench_mark].index
drop_movie_index

Index([   1,    2,    4,    7,    9,   10,   11,   12,   13,   14,
       ...
       4480, 4481, 4486, 4487, 4491, 4494, 4495, 4497, 4498, 4499],
      dtype='int64', name='Movie_id', length=2699)

In [19]:
len(drop_movie_index)

2699

In [20]:
cust_rew_count = netflix_dataset['Cust_id'].value_counts()
cust_rew_count

Cust_id
305344     4467
387418     4422
2439493    4195
1664010    4019
2118461    3769
           ... 
1300341       1
2550360       1
11848         1
930788        1
594210        1
Name: count, Length: 470758, dtype: int64

In [21]:
bench_mark_cus=round(cust_rew_count.quantile(.6),0)
bench_mark_cus

36.0

In [22]:
drop_cust_index=cust_rew_count[cust_rew_count<bench_mark_cus].index
drop_cust_index


Index([2194851,  600295, 1739398, 1157368,  532108, 2157249,  256134,  640441,
       1272324, 1346990,
       ...
       1969065,  899932,  611596, 2147176,  811650, 1300341, 2550360,   11848,
        930788,  594210],
      dtype='int64', name='Cust_id', length=282042)

In [23]:
netflix_dataset=netflix_dataset[~netflix_dataset['Movie_id'].isin(drop_movie_index)]
netflix_dataset=netflix_dataset[~netflix_dataset['Cust_id'].isin(drop_cust_index)]

In [24]:
netflix_dataset

Cust_id  Ratings  Movie_id
696        712664      5.0         3
697       1331154      4.0         3
698       2632461      3.0         3
699         44937      5.0         3
700        656399      4.0         3
...           ...      ...       ...
24056842  1055714      5.0      4496
24056843  2643029      4.0      4496
24056844   267802      4.0      4496
24056845  1559566      3.0      4496
24056846   293198      3.0      4496

[19695836 rows x 3 columns]

In [25]:
movie_title=pd.read_csv('/content/drive/MyDrive/Netflix_Dataset/Copy of movie_titles.csv', encoding='ISO-8859-1' ,header=None, names=['Movie_id','Year','Name'], usecols=[0,1,2])

In [26]:
movie_title

Movie_id    Year                                               Name
0             1  2003.0                                    Dinosaur Planet
1             2  2004.0                         Isle of Man TT 2004 Review
2             3  1997.0                                          Character
3             4  1994.0                       Paula Abdul's Get Up & Dance
4             5  2004.0                           The Rise and Fall of ECW
...         ...     ...                                                ...
17765     17766  2002.0  Where the Wild Things Are and Other Maurice Se...
17766     17767  2004.0                  Fidel Castro: American Experience
17767     17768  2000.0                                              Epoch
17768     17769  2003.0                                        The Company
17769     17770  2003.0                                       Alien Hunter

[17770 rows x 3 columns]

In [27]:
#!pip install numpy==1.23.5

In [28]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl


In [29]:
from surprise import SVD, Reader,Dataset
from surprise.model_selection import cross_validate

In [30]:
reader=Reader()

In [31]:
data=Dataset.load_from_df(netflix_dataset[['Movie_id','Cust_id','Ratings']][:100000],reader)

In [32]:
data

In [33]:
model=SVD()

In [34]:
# cross validation: split the data into train and test and perform training and testing in different patterns.
cross_validate(model,data,measures=['RMSE'],cv=3)

{'test_rmse': array([1.0200505 , 1.02066152, 1.01668711]),
 'fit_time': (2.1562395095825195, 1.6461000442504883, 3.941816568374634),
 'test_time': (0.28305935859680176, 0.3411903381347656, 1.7050423622131348)}

In [35]:
user_rating=netflix_dataset[netflix_dataset['Cust_id']==1331154]
user_rating

Cust_id  Ratings  Movie_id
697       1331154      4.0         3
5178      1331154      4.0         8
31460     1331154      3.0        18
92840     1331154      4.0        30
224761    1331154      3.0        44
...           ...      ...       ...
23439584  1331154      4.0      4389
23546489  1331154      2.0      4402
23649431  1331154      4.0      4432
23844441  1331154      3.0      4472
23985561  1331154      4.0      4488

[253 rows x 3 columns]

In [36]:
user_1331154=movie_title.copy()
user_1331154

Movie_id    Year                                               Name
0             1  2003.0                                    Dinosaur Planet
1             2  2004.0                         Isle of Man TT 2004 Review
2             3  1997.0                                          Character
3             4  1994.0                       Paula Abdul's Get Up & Dance
4             5  2004.0                           The Rise and Fall of ECW
...         ...     ...                                                ...
17765     17766  2002.0  Where the Wild Things Are and Other Maurice Se...
17766     17767  2004.0                  Fidel Castro: American Experience
17767     17768  2000.0                                              Epoch
17768     17769  2003.0                                        The Company
17769     17770  2003.0                                       Alien Hunter

[17770 rows x 3 columns]

In [37]:
user_1331154=user_1331154[~user_1331154['Movie_id'].isin(drop_movie_index)]
user_1331154

Movie_id    Year                                               Name
2             3  1997.0                                          Character
4             5  2004.0                           The Rise and Fall of ECW
5             6  1997.0                                               Sick
7             8  2004.0                         What the #$*! Do We Know!?
15           16  1996.0                                          Screamers
...         ...     ...                                                ...
17765     17766  2002.0  Where the Wild Things Are and Other Maurice Se...
17766     17767  2004.0                  Fidel Castro: American Experience
17767     17768  2000.0                                              Epoch
17768     17769  2003.0                                        The Company
17769     17770  2003.0                                       Alien Hunter

[15071 rows x 3 columns]

In [38]:
user_1331154['Estimated']=user_1331154['Movie_id'].apply(lambda x:model.predict(1331154,x).est)
user_1331154

<ipython-input-38-204300570>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_1331154['Estimated']=user_1331154['Movie_id'].apply(lambda x:model.predict(1331154,x).est)


Movie_id    Year                                               Name  \
2             3  1997.0                                          Character   
4             5  2004.0                           The Rise and Fall of ECW   
5             6  1997.0                                               Sick   
7             8  2004.0                         What the #$*! Do We Know!?   
15           16  1996.0                                          Screamers   
...         ...     ...                                                ...   
17765     17766  2002.0  Where the Wild Things Are and Other Maurice Se...   
17766     17767  2004.0                  Fidel Castro: American Experience   
17767     17768  2000.0                                              Epoch   
17768     17769  2003.0                                        The Company   
17769     17770  2003.0                                       Alien Hunter   

       Estimated  
2       3.583707  
4       3.583707  
5       3.583707  
7       3.583707  
15      3.583707  
...          ...  
17765   3.583707  
17766   3.583707  
17767   3.583707  
17768   3.583707  
17769   3.583707  

[15071 rows x 4 columns]

In [39]:
user_1331154=user_1331154.sort_values('Estimated',ascending=False)

In [40]:
user_1331154.head()

Movie_id    Year                                Name  Estimated
13874     13875  1982.0    Gilbert and Sullivan: The Mikado   3.891775
8814       8815  1992.0                  Chaplin: The Movie   3.855929
13510     13511  1993.0              Much Ado About Nothing   3.846859
6459       6460  2005.0  The Game: The Documentary: The DVD   3.835153
10373     10374  1997.0             Goosebumps: Scary House   3.796245